<a href="https://colab.research.google.com/github/fjpa121197/ImageCLEF2021/blob/main/Multi_label_classification/ImageCLEF2021_Multi_label_Classification_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from zipfile import ZipFile
os.environ['KAGGLE_USERNAME'] = "#####" # username from the json file
os.environ['KAGGLE_KEY'] = "#####" # key from the json file
!kaggle datasets download -d fjpa121197/imageclefmed-concept-detection-2021
!kaggle datasets download -d fjpa121197/training-images-concepts-by-semantic-type
!kaggle datasets download -d fjpa121197/imageclefmed2021 # api copied from kaggle !!!!!!!CHANGE API COMMAND TO THE NEW DATASET

imageclefmed-concept-detection-2021.zip: Skipping, found more recently modified local copy (use --force to force download)
training-images-concepts-by-semantic-type.zip: Skipping, found more recently modified local copy (use --force to force download)
imageclefmed2021.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Unzip 2021 data
clef2021 = "/content/imageclefmed-concept-detection-2021.zip"
with ZipFile(clef2021, 'r') as zip:
  zip.extractall()
  print('done with 2021 image dataset')

clef2021_concepts = "/content/training-images-concepts-by-semantic-type.zip"
with ZipFile(clef2021_concepts, 'r') as zip:
  zip.extractall()
  print('done with 2021 concepts dataset')

# Unzip 2020 data
clef2020 = "/content/imageclefmed2021.zip"
with ZipFile(clef2020, 'r') as zip:
  zip.extractall()
  print('done with 2020 image dataset')

done with 2021 image dataset
done with 2021 concepts dataset
done with 2020 image dataset


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import json
import pickle
import os
import random

# Multi-label Classification without autoencoders and Densenet-121

## Using 2021 data only (all semantic type labels)

In [ ]:
# Utils functions
def extract_concepts(root_paths, image_id_concepts_dict = dict()):

    for idx, name in enumerate(root_paths):
      with open(name, "r", encoding= 'utf-8-sig') as f:
        reader = csv.reader(f, delimiter = '\t')
        
        if name == '/content/Training_Set_Concepts.csv':
          image_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'
        else:
          image_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'
        for i, line in enumerate(reader):
          if len(line[1]) < 1:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = []
          else:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict


def transform_images(path_to_image):
  #path_to_image = os.path.join(training_images_dir, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size= (224,224))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/Training_Set_Concepts.csv','/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation_Set_Concepts.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts)

In [ ]:
X = []
Y = []
images_ids = []

In [ ]:
for image in image_id_concepts_dict.keys():
  X.append(transform_images(image))
  Y.append(image_id_concepts_dict[image])
  images_ids.append(image.split("/")[-1].split("."))

In [ ]:
X = np.array(X)
Y = np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
# Use a multilabelbinarizer to transform the concepts into a binary format for training
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)
print(len(mlb.classes_))

1585


In [ ]:
X, X_test, y, y_test = train_test_split(X, Y, test_size = 0.1, shuffle = True, random_state = 14)
ids_images_train, ids_images_test = train_test_split(images_ids, test_size = 0.1, shuffle=True, random_state = 14) 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 14) 
ids_images_train_train, ids_images_val = train_test_split(ids_images_train, test_size = 0.2, shuffle=True, random_state = 14)

In [ ]:
X_train.shape
y_train.shape

(2344, 1585)

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid', name = 'prediction_layer')(x)


mlcf_model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x)

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
mlcf_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator to be used for training
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)
train_generator = data_generator.flow(X_train, y_train, batch_size = 32, subset = 'training', seed = 14)
val_generator = data_generator.flow(X_train, y_train, batch_size = 32, subset = 'validation', seed = 14)

In [ ]:
history = mlcf_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
59/59 [==============================] - 24s 292ms/step - loss: 0.5244 - acc: 0.0023 - val_loss: 0.1377 - val_acc: 0.0598
Epoch 2/100
59/59 [==============================] - 15s 257ms/step - loss: 0.1088 - acc: 0.1129 - val_loss: 0.0577 - val_acc: 0.2030
Epoch 3/100
59/59 [==============================] - 15s 258ms/step - loss: 0.0507 - acc: 0.2133 - val_loss: 0.0363 - val_acc: 0.2457
Epoch 4/100
59/59 [==============================] - 15s 259ms/step - loss: 0.0331 - acc: 0.2664 - val_loss: 0.0270 - val_acc: 0.2756
Epoch 5/100
59/59 [==============================] - 15s 261ms/step - loss: 0.0253 - acc: 0.2808 - val_loss: 0.0221 - val_acc: 0.2927
Epoch 6/100
59/59 [==============================] - 15s 261ms/step - loss: 0.0209 - acc: 0.2989 - val_loss: 0.0191 - val_acc: 0.2927
Epoch 7/100
59/59 [==============================] - 15s 261ms/step - loss: 0.0187 - acc: 0.2895 - val_loss: 0.0171 - val_acc: 0.2927
Epoch 8/100
59/59 [==============================] - 15s 259ms

In [ ]:
layer_names = [layer.name for layer in mlcf_model.layers]

In [ ]:
layer_idx = layer_names.index('conv5_block1_0_bn')

In [ ]:
for layer in mlcf_model.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=new_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
mlcf_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
history_fined = mlcf_model.fit(train_generator, epochs = 150, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks, initial_epoch = history.epoch[-1])

Epoch 53/150
59/59 [==============================] - 28s 323ms/step - loss: 0.0084 - acc: 0.3801 - val_loss: 0.0104 - val_acc: 0.3590
Epoch 54/150
59/59 [==============================] - 17s 287ms/step - loss: 0.0083 - acc: 0.3578 - val_loss: 0.0105 - val_acc: 0.3568
Epoch 55/150
59/59 [==============================] - 17s 289ms/step - loss: 0.0080 - acc: 0.3764 - val_loss: 0.0104 - val_acc: 0.3568
Epoch 56/150
59/59 [==============================] - 17s 290ms/step - loss: 0.0077 - acc: 0.3731 - val_loss: 0.0103 - val_acc: 0.3568
Epoch 57/150
59/59 [==============================] - 17s 291ms/step - loss: 0.0077 - acc: 0.3940 - val_loss: 0.0103 - val_acc: 0.3590
Epoch 58/150
59/59 [==============================] - 17s 291ms/step - loss: 0.0077 - acc: 0.3875 - val_loss: 0.0102 - val_acc: 0.3590


## Using 2021 data only (only with dp labels)

In [ ]:
# Utils functions
def extract_concepts(root_paths, image_id_concepts_dict = dict()):

    for idx, name in enumerate(root_paths):
      with open(name, "r", encoding= 'utf-8-sig') as f:
        reader = csv.reader(f, delimiter = '\t')
        
        if name == '/content/training-images-concepts-by-semantic/concepts-file/training-concepts-dp-only.csv':
          image_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'
        else:
          image_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'
        for i, line in enumerate(reader):
          if len(line[1]) < 1:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = []
          else:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict


def transform_images(path_to_image):
  #path_to_image = os.path.join(training_images_dir, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size= (224,224))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/training-images-concepts-by-semantic/concepts-file/training-concepts-dp-only.csv',
                    '/content/val-concepts-dp-only.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts)

In [ ]:
# Define array where images array will be saved for training, and where the concepts for each image will be saved (Y)
X = []
Y = []

In [ ]:
# Load images

for image in image_id_concepts_dict.keys():
  X.append(transform_images(image))
  Y.append(image_id_concepts_dict[image])

In [ ]:
# Transform arrays to numpy arrays
X = np.array(X)
Y = np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Use a multilabelbinarizer to transform the concepts into a binary format for training
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)
print(len(mlb.classes_))

110


Model definition

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid', name = 'prediction_layer')(x)


dp_model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x)

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 10, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
dp_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator to be used for training
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)
train_generator = data_generator.flow(X, Y, batch_size = 32, subset = 'training', seed = 14)
val_generator = data_generator.flow(X, Y, batch_size = 32, subset = 'validation', seed = 14)

In [ ]:
history = dp_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
82/82 [==============================] - 33s 315ms/step - loss: 0.3821 - acc: 0.1387 - val_loss: 0.0297 - val_acc: 0.6390
Epoch 2/100
82/82 [==============================] - 21s 257ms/step - loss: 0.0260 - acc: 0.6836 - val_loss: 0.0216 - val_acc: 0.7619
Epoch 3/100
82/82 [==============================] - 21s 258ms/step - loss: 0.0199 - acc: 0.7961 - val_loss: 0.0191 - val_acc: 0.8003
Epoch 4/100
82/82 [==============================] - 21s 258ms/step - loss: 0.0169 - acc: 0.8443 - val_loss: 0.0176 - val_acc: 0.8203
Epoch 5/100
82/82 [==============================] - 21s 260ms/step - loss: 0.0156 - acc: 0.8549 - val_loss: 0.0168 - val_acc: 0.8295
Epoch 6/100
82/82 [==============================] - 21s 259ms/step - loss: 0.0141 - acc: 0.8641 - val_loss: 0.0161 - val_acc: 0.8341
Epoch 7/100
82/82 [==============================] - 21s 259ms/step - loss: 0.0138 - acc: 0.8740 - val_loss: 0.0156 - val_acc: 0.8372
Epoch 8/100
82/82 [==============================] - 21s 259ms

In [ ]:
layer_names = [layer.name for layer in default_densenet.layers]

In [ ]:
layer_idx = layer_names.index('conv5_block1_0_bn')

In [ ]:
for layer in default_densenet.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=new_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
dp_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
history_fined = dp_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
82/82 [==============================] - 34s 319ms/step - loss: 0.0080 - acc: 0.9060 - val_loss: 0.0124 - val_acc: 0.8694
Epoch 2/100
82/82 [==============================] - 24s 288ms/step - loss: 0.0069 - acc: 0.9212 - val_loss: 0.0123 - val_acc: 0.8710
Epoch 3/100
82/82 [==============================] - 24s 289ms/step - loss: 0.0063 - acc: 0.9254 - val_loss: 0.0122 - val_acc: 0.8740
Epoch 4/100
82/82 [==============================] - 24s 289ms/step - loss: 0.0057 - acc: 0.9271 - val_loss: 0.0121 - val_acc: 0.8771
Epoch 5/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0054 - acc: 0.9317 - val_loss: 0.0119 - val_acc: 0.8802
Epoch 6/100
82/82 [==============================] - 24s 290ms/step - loss: 0.0051 - acc: 0.9317 - val_loss: 0.0119 - val_acc: 0.8802
Epoch 7/100
82/82 [==============================] - 24s 290ms/step - loss: 0.0045 - acc: 0.9373 - val_loss: 0.0118 - val_acc: 0.8802
Epoch 8/100
82/82 [==============================] - 24s 290ms

In [ ]:
# Now lets use the validation images to create a submission file and evaluate it.
val_x_predict = []
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img) # Transform image to array of shape (input_shape)
  img = tf.keras.applications.densenet.preprocess_input(img) # This preprocess_input normalizes the pixel values based on imagenet dataset and rescale to a 0-1 values.
  val_x_predict.append(img)

100%|██████████| 500/500 [00:04<00:00, 120.93it/s]


In [ ]:
val_x_predict = np.array(val_x_predict) # A numpy array is needed as input for the model
val_predictions = dp_model.predict(val_x_predict)

In [ ]:
# Use previous threshold with better f1-score
val_predictions[val_predictions>=0.4] = 1
val_predictions[val_predictions<0.4] = 0
val_labels_predicted = mlb.inverse_transform(val_predictions) #This returns a list of tuples

In [ ]:
# The concept(s) are needed as strings separated by ; if applicable
val_labels_united = []
for prediction in val_labels_predicted:
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append(str_concepts[0:-1])

# The image id needs to be included in the submission
val_images_ids = []
for image in tqdm(os.listdir(validation_images_path), position= 0):
  val_images_ids.append(image.split('.')[0])

# Pass to df  to use to_csv function
predictions_df = pd.DataFrame({'image_ids': val_images_ids})
predictions_df['predictions'] = pd.Series(val_labels_united)
predictions_df.to_csv('/content/mlcf-best-model-dp-only-dp-labels.csv', index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab

100%|██████████| 500/500 [00:00<00:00, 190615.52it/s]


In [ ]:
dp_model.save('dp-classifier-partial-unfreeze-threshold40-use-for-predictions.h5')

In [ ]:
with open("mlb_dp_classifier.pkl", 'wb') as f:
    pickle.dump(mlb, f)

## Using 2021 data only (only with bpo)

In [ ]:
# Utils functions
def extract_concepts(root_paths, image_id_concepts_dict = dict()):

    for idx, name in enumerate(root_paths):
      with open(name, "r", encoding= 'utf-8-sig') as f:
        reader = csv.reader(f, delimiter = '\t')
        
        if name == '/content/training-images-concepts-by-semantic/concepts-file/training-concepts-bpo-only.csv':
          image_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'
        else:
          image_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'
        for i, line in enumerate(reader):
          if len(line[1]) < 1:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = []
          else:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict


def transform_images(path_to_image):
  #path_to_image = os.path.join(training_images_dir, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size= (224,224))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/training-images-concepts-by-semantic/concepts-file/training-concepts-bpo-only.csv',
                    '/content/val-concepts-bpo-only.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts)

In [ ]:
# Define array where images array will be saved for training, and where the concepts for each image will be saved (Y)
X = []
Y = []

In [ ]:
# Load images
for image in image_id_concepts_dict.keys():
  X.append(transform_images(image))
  Y.append(image_id_concepts_dict[image])

In [ ]:
# Transform arrays to numpy arrays
X = np.array(X)
Y = np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Use a multilabelbinarizer to transform the concepts into a binary format for training
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)
print(len(mlb.classes_))

478


Model definition

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid', name = 'prediction_layer')(x)


bpo_model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x)

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
bpo_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator to be used for training
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)
train_generator = data_generator.flow(X, Y, batch_size = 32, subset = 'training', seed = 14)
val_generator = data_generator.flow(X, Y, batch_size = 32, subset = 'validation', seed = 14)

In [ ]:
history = bpo_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
82/82 [==============================] - 63s 321ms/step - loss: 0.4465 - acc: 3.2262e-04 - val_loss: 0.0152 - val_acc: 0.0031
Epoch 2/100
82/82 [==============================] - 21s 258ms/step - loss: 0.0138 - acc: 0.0024 - val_loss: 0.0106 - val_acc: 0.0123
Epoch 3/100
82/82 [==============================] - 21s 259ms/step - loss: 0.0118 - acc: 0.0182 - val_loss: 0.0101 - val_acc: 0.0108
Epoch 4/100
82/82 [==============================] - 21s 260ms/step - loss: 0.0112 - acc: 0.0259 - val_loss: 0.0100 - val_acc: 0.0138
Epoch 5/100
82/82 [==============================] - 21s 260ms/step - loss: 0.0109 - acc: 0.0268 - val_loss: 0.0099 - val_acc: 0.0215
Epoch 6/100
82/82 [==============================] - 21s 260ms/step - loss: 0.0106 - acc: 0.0241 - val_loss: 0.0098 - val_acc: 0.0246
Epoch 7/100
82/82 [==============================] - 21s 260ms/step - loss: 0.0106 - acc: 0.0345 - val_loss: 0.0097 - val_acc: 0.0338
Epoch 8/100
82/82 [==============================] - 21s 2

In [ ]:
layer_names = [layer.name for layer in default_densenet.layers]

In [ ]:
layer_idx = layer_names.index('conv5_block1_0_bn')

In [ ]:
for layer in default_densenet.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=new_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
bpo_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
history_fined = bpo_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
82/82 [==============================] - 35s 321ms/step - loss: 0.0056 - acc: 0.3342 - val_loss: 0.0076 - val_acc: 0.1736
Epoch 2/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0052 - acc: 0.3801 - val_loss: 0.0076 - val_acc: 0.1813
Epoch 3/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0050 - acc: 0.3898 - val_loss: 0.0076 - val_acc: 0.1720
Epoch 4/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0048 - acc: 0.4468 - val_loss: 0.0075 - val_acc: 0.1720
Epoch 5/100
82/82 [==============================] - 24s 294ms/step - loss: 0.0046 - acc: 0.4490 - val_loss: 0.0075 - val_acc: 0.1751
Epoch 6/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0044 - acc: 0.4491 - val_loss: 0.0075 - val_acc: 0.1751
Epoch 7/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0043 - acc: 0.4544 - val_loss: 0.0075 - val_acc: 0.1828
Epoch 8/100
82/82 [==============================] - 24s 295ms

In [ ]:
# Now lets use the validation images to create a submission file and evaluate it.
val_x_predict = []
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img) # Transform image to array of shape (input_shape)
  img = tf.keras.applications.densenet.preprocess_input(img) # This preprocess_input normalizes the pixel values based on imagenet dataset and rescale to a 0-1 values.
  val_x_predict.append(img)

100%|██████████| 500/500 [00:04<00:00, 112.00it/s]


In [ ]:
val_x_predict = np.array(val_x_predict) # A numpy array is needed as input for the model
val_predictions = bpo_model.predict(val_x_predict)

In [ ]:
# Use previous threshold with better f1-score
val_predictions[val_predictions>=0.1] = 1
val_predictions[val_predictions<0.1] = 0
val_labels_predicted = mlb.inverse_transform(val_predictions) #This returns a list of tuples

In [ ]:
# The concept(s) are needed as strings separated by ; if applicable
val_labels_united = []
for prediction in val_labels_predicted:
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append(str_concepts[0:-1])

# The image id needs to be included in the submission
val_images_ids = []
for image in tqdm(os.listdir(validation_images_path), position= 0):
  val_images_ids.append(image.split('.')[0])

# Pass to df  to use to_csv function
predictions_df = pd.DataFrame({'image_ids': val_images_ids})
predictions_df['predictions'] = pd.Series(val_labels_united)
predictions_df.to_csv('/content/mlcf-best-model-bpo-only-bpo-labels.csv', index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab

100%|██████████| 500/500 [00:00<00:00, 339729.79it/s]


In [ ]:
bpo_model.save('bpo-classifier-partial-unfreeze-threshold1-use-for-predictions.h5')

In [ ]:
with open("mlb_bpo_classifier.pkl", 'wb') as f:
    pickle.dump(mlb, f)

In [ ]:
dp_predictions = pd.read_csv('/content/mlcf-best-model-dp-only-dp-labels.csv', header=None, delimiter='\t', names=['ImageId', 'dp_tags'])

In [ ]:
bpo_predictions = pd.read_csv('/content/mlcf-best-model-bpo-only-bpo-labels.csv', header=None, delimiter='\t', names=['ImageId', 'bpo_tags'])

In [ ]:
dp_bpo_merged = pd.merge(dp_predictions,bpo_predictions, on='ImageId')

In [ ]:
dp_bpo_merged['dp_bpo_tags'] = dp_bpo_merged[dp_bpo_merged.columns[1:]].apply(lambda row: ';'.join(row.dropna()), axis = 1)

In [ ]:
dp_bpo_merged.to_csv('/content/mlcf-best-models-dp-bpo-labels.csv', index= False, sep ='\t', header= False, columns=['ImageId','dp_bpo_tags'])

Merging both the dp and bpo predictions gives a 0.5808 f1 score 

## Using 2021 data only (only with blr)

In [ ]:
# Utils functions
def extract_concepts(root_paths, image_id_concepts_dict = dict()):

    for idx, name in enumerate(root_paths):
      with open(name, "r", encoding= 'utf-8-sig') as f:
        reader = csv.reader(f, delimiter = '\t')
        
        if name == '/content/training-images-concepts-by-semantic/concepts-file/training-concepts-blr-only.csv':
          image_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'
        else:
          image_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'
        for i, line in enumerate(reader):
          if len(line[1]) < 1:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = []
          else:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict


def transform_images(path_to_image):
  #path_to_image = os.path.join(training_images_dir, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size= (224,224))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/training-images-concepts-by-semantic/concepts-file/training-concepts-blr-only.csv',
                    '/content/val-concepts-blr-only.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts)

In [ ]:
# Define array where images array will be saved for training, and where the concepts for each image will be saved (Y)
X = []
Y = []

In [ ]:
# Load images
for image in image_id_concepts_dict.keys():
  X.append(transform_images(image))
  Y.append(image_id_concepts_dict[image])

In [ ]:
# Transform arrays to numpy arrays
X = np.array(X)
Y = np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Use a multilabelbinarizer to transform the concepts into a binary format for training
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)
print(len(mlb.classes_))

148


Model definition

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid', name = 'prediction_layer')(x)


blr_model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x)

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
blr_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator to be used for training
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)
train_generator = data_generator.flow(X, Y, batch_size = 32, subset = 'training', seed = 14)
val_generator = data_generator.flow(X, Y, batch_size = 32, subset = 'validation', seed = 14)

In [ ]:
history = blr_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
82/82 [==============================] - 51s 158ms/step - loss: 0.3872 - acc: 0.0118 - val_loss: 0.0140 - val_acc: 0.0722
Epoch 2/100
82/82 [==============================] - 9s 111ms/step - loss: 0.0124 - acc: 0.1063 - val_loss: 0.0097 - val_acc: 0.1306
Epoch 3/100
82/82 [==============================] - 9s 113ms/step - loss: 0.0102 - acc: 0.1475 - val_loss: 0.0091 - val_acc: 0.1260
Epoch 4/100
82/82 [==============================] - 9s 113ms/step - loss: 0.0103 - acc: 0.1251 - val_loss: 0.0089 - val_acc: 0.2473
Epoch 5/100
82/82 [==============================] - 9s 115ms/step - loss: 0.0094 - acc: 0.1969 - val_loss: 0.0087 - val_acc: 0.1690
Epoch 6/100
82/82 [==============================] - 9s 115ms/step - loss: 0.0093 - acc: 0.1590 - val_loss: 0.0086 - val_acc: 0.1521
Epoch 7/100
82/82 [==============================] - 10s 116ms/step - loss: 0.0089 - acc: 0.1882 - val_loss: 0.0085 - val_acc: 0.2012
Epoch 8/100
82/82 [==============================] - 10s 117ms/step

In [ ]:
layer_names = [layer.name for layer in default_densenet.layers]

In [ ]:
layer_idx = layer_names.index('conv5_block1_0_bn')

In [ ]:
for layer in default_densenet.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=new_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
blr_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
history_fined = blr_model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
82/82 [==============================] - 19s 151ms/step - loss: 0.0089 - acc: 0.1989 - val_loss: 0.0084 - val_acc: 0.3164
Epoch 2/100
82/82 [==============================] - 11s 131ms/step - loss: 0.0083 - acc: 0.2549 - val_loss: 0.0083 - val_acc: 0.2596
Epoch 3/100
82/82 [==============================] - 11s 133ms/step - loss: 0.0076 - acc: 0.2318 - val_loss: 0.0082 - val_acc: 0.2273
Epoch 4/100
82/82 [==============================] - 11s 132ms/step - loss: 0.0075 - acc: 0.2314 - val_loss: 0.0081 - val_acc: 0.2058
Epoch 5/100
82/82 [==============================] - 11s 131ms/step - loss: 0.0071 - acc: 0.2302 - val_loss: 0.0080 - val_acc: 0.2181
Epoch 6/100
82/82 [==============================] - 11s 130ms/step - loss: 0.0069 - acc: 0.2466 - val_loss: 0.0079 - val_acc: 0.2089


In [ ]:
# Now lets use the validation images to create a submission file and evaluate it.
val_x_predict = []
val_x_ids = []
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img) # Transform image to array of shape (input_shape)
  img = tf.keras.applications.densenet.preprocess_input(img) # This preprocess_input normalizes the pixel values based on imagenet dataset and rescale to a 0-1 values.
  val_x_predict.append(img)
  val_x_ids.append(image.split('.')[0])

100%|██████████| 500/500 [00:03<00:00, 137.20it/s]


In [ ]:
val_x_predict = np.array(val_x_predict) # A numpy array is needed as input for the model
val_predictions = blr_model.predict(val_x_predict)

In [ ]:
val_predictions[0]

array([9.9230539e-03, 6.5643666e-04, 3.9436316e-04, 2.0860463e-04,
       5.6231231e-04, 4.0287017e-05, 3.5653258e-04, 2.1965045e-03,
       3.2209608e-04, 4.7119358e-03, 1.3389805e-02, 1.6430472e-03,
       2.5318735e-03, 1.7232983e-04, 8.8644703e-04, 4.8705048e-04,
       9.5687853e-04, 5.0304802e-03, 1.2588501e-04, 1.7467537e-04,
       2.1316929e-04, 2.6105015e-04, 6.9832487e-04, 2.5129046e-03,
       3.6980954e-04, 3.5331075e-04, 1.4628364e-04, 6.3982472e-04,
       3.8793168e-04, 5.7095318e-04, 3.7747432e-04, 2.2042077e-03,
       2.3669214e-04, 8.7233551e-04, 8.7851065e-04, 5.1259104e-04,
       3.2021333e-05, 2.7847196e-05, 6.6841755e-04, 2.1642186e-04,
       1.7088711e-04, 1.7257048e-03, 3.7882547e-04, 8.4804968e-05,
       1.6336015e-04, 1.4383319e-03, 1.3294074e-03, 1.0389987e-03,
       6.0987956e-04, 1.8747467e-04, 4.1042283e-04, 3.3807885e-04,
       3.9068301e-04, 1.0203137e-03, 1.5733196e-04, 7.0605056e-05,
       5.3868396e-04, 6.7578820e-03, 1.1992079e-03, 1.1883827e

In [ ]:
# Use previous threshold with better f1-score
val_predictions[val_predictions>=0.05] = 1
val_predictions[val_predictions<0.05] = 0
val_labels_predicted = mlb.inverse_transform(val_predictions) #This returns a list of tuples

In [ ]:
# The concept(s) are needed as strings separated by ; if applicable
val_labels_united = []
for idx,prediction in enumerate(val_labels_predicted):
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append([val_x_ids[idx],str_concepts[0:-1]])


# Pass to df  to use to_csv function
predictions_df = pd.DataFrame(val_labels_united, columns = ['ImageId','concepts'])
predictions_df.to_csv('/content/mlcf-best-model-blr-only-blr-labels.csv', index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab

Merging the previous outputs (dp and bpo) with blr output lower the f1-score on the validation set to 0.567

## Using 2021 + 2020 data (without autoencoders) and only with diagnostic procedure

In [ ]:
path_to_concepts = ['/content/images-combined-dp-only.csv']

In [ ]:
def extract_concepts_all(root_paths, image_id_concepts_dict = dict()):
    for idx, name in enumerate(root_paths):
        with open(name, "r", encoding= 'utf-8-sig') as f:
          reader = csv.reader(f, delimiter = '\t')
          for i, line in enumerate(reader):
            if len(line[1])> 1:
              image_id_concepts_dict[line[0]] = list(line[1].split(";"))
            else:
              image_id_concepts_dict[line[0]] = []
    
    return image_id_concepts_dict

In [ ]:
#Extract concepts for the multiple concept files
image_id_concepts_dict = extract_concepts_all(path_to_concepts)

In [ ]:
# Since we are working with around 9K images. We will only load the images absolute path and the concepts to a dataframe and then use a generator to load them during training.
# Here, we will create a dataframe with the images path
all_images_path = []
# Training images
for image in tqdm(image_id_concepts_dict.keys(), position = 0):
  all_images_path.append([image])
df_all_images = pd.DataFrame(all_images_path, columns=['image_path'])

100%|██████████| 83979/83979 [00:00<00:00, 206828.24it/s]


In [ ]:
concepts = []
for image in tqdm(image_id_concepts_dict.keys(), position=0):
  concepts.append(image_id_concepts_dict[image])

100%|██████████| 83979/83979 [00:00<00:00, 1116411.64it/s]


In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
# Since we will use flow_from_dataframe in the training, we put both the images absolute path and the encoded labels
df_use_densenet = pd.concat([df_all_images, pd.DataFrame(np.array(mlb.fit_transform(concepts)))], axis=1)

In [ ]:
len(mlb.classes_)

230

In [ ]:
concepts_binarized = np.array(mlb.transform(concepts))

In [ ]:
# Train split dataset, because a portion is needed to set the threshold (to see if to assign the concept or not) and another portion to see the overall f1-score
df_use_train, df_test = train_test_split(df_use_densenet, test_size = 0.05, shuffle = True, random_state = 14) # test will be used to get a final f1-score
df_train, df_val = train_test_split(df_use_train, test_size=0.05, shuffle=True, random_state=14)

In [ ]:
concepts_binarized_use_train, concepts_binarized_test = train_test_split(concepts_binarized, test_size = 0.05, shuffle = True, random_state = 14)
concepts_binarized_train, concepts_binarized_val = train_test_split(concepts_binarized_use_train, test_size = 0.05, shuffle = True, random_state = 14)

In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(75791, 231)
(3989, 231)
(4199, 231)


In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid', name = 'prediction_layer')(x)


model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x) # Final model to be trained

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 3, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Train generator
train_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=64, shuffle=True, seed=14, subset='training')

# Validation generator
val_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=64, shuffle=True, seed=14, subset='validation')

Found 60633 validated image filenames.
Found 15158 validated image filenames.


In [ ]:
# Model training (only the classification layers that have been added)
history = model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks, batch_size = 64)

Epoch 1/100
948/948 [==============================] - 557s 576ms/step - loss: 0.1587 - acc: 0.3079 - val_loss: 0.0239 - val_acc: 0.6876
Epoch 2/100
948/948 [==============================] - 537s 567ms/step - loss: 0.0223 - acc: 0.7105 - val_loss: 0.0197 - val_acc: 0.7579
Epoch 3/100
948/948 [==============================] - 534s 563ms/step - loss: 0.0192 - acc: 0.7639 - val_loss: 0.0181 - val_acc: 0.7954
Epoch 4/100
948/948 [==============================] - 530s 559ms/step - loss: 0.0179 - acc: 0.7937 - val_loss: 0.0172 - val_acc: 0.7977
Epoch 5/100
948/948 [==============================] - 525s 554ms/step - loss: 0.0167 - acc: 0.8015 - val_loss: 0.0166 - val_acc: 0.8177
Epoch 6/100
948/948 [==============================] - 522s 551ms/step - loss: 0.0163 - acc: 0.8151 - val_loss: 0.0162 - val_acc: 0.8173
Epoch 7/100
948/948 [==============================] - 528s 557ms/step - loss: 0.0159 - acc: 0.8162 - val_loss: 0.0158 - val_acc: 0.8212
Epoch 8/100
948/948 [====================

In [ ]:
layer_names = [layer.name for layer in default_densenet.layers]
layer_idx = layer_names.index('conv4_block7_0_bn')

In [ ]:
for layer in default_densenet.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Model training (of the entire model)
history_fined = model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
948/948 [==============================] - 718s 745ms/step - loss: 0.0143 - acc: 0.8297 - val_loss: 0.0133 - val_acc: 0.8613
Epoch 2/100
948/948 [==============================] - 704s 742ms/step - loss: 0.0110 - acc: 0.8639 - val_loss: 0.0133 - val_acc: 0.8652
Epoch 3/100
948/948 [==============================] - 701s 739ms/step - loss: 0.0089 - acc: 0.8744 - val_loss: 0.0133 - val_acc: 0.8385
Epoch 4/100
948/948 [==============================] - 701s 740ms/step - loss: 0.0062 - acc: 0.8853 - val_loss: 0.0146 - val_acc: 0.8553
Epoch 5/100
948/948 [==============================] - 704s 742ms/step - loss: 0.0046 - acc: 0.8874 - val_loss: 0.0159 - val_acc: 0.8560


In [ ]:
val_gen_pred = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Validation data generator for threshold tunning

val_generator_pred = val_gen_pred.flow_from_dataframe(df_val,x_col='image_path', y_col=df_val.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=64)

Found 3989 validated image filenames.


In [ ]:
# Since the predictions made by the model is a list of probabilities that a particular concept (from the one seen in training) is present, i.e [0.01, 0.5,...]
# It is needed to set a threshold for prob (0-1) that maximizes this f1 score.
probs = np.arange(0.05,1.0,0.05)
scores = []
for prob in probs:
  preds = model.predict(val_generator_pred) # Use the validation set
  preds[preds>=prob] = 1
  preds[preds<prob] = 0
  scores.append(tuple((f1_score(concepts_binarized_val, preds, average="micro"),prob)))
  print(tuple((f1_score(concepts_binarized_val, preds, average="micro"),prob)))

(0.1405668611328035, 0.05)
(0.16292392300641614, 0.1)
(0.1763962597927723, 0.15000000000000002)
(0.1913403162319034, 0.2)
(0.18704436575713393, 0.25)
(0.19669091427756383, 0.3)
(0.1872724625018203, 0.35000000000000003)
(0.1949955581877406, 0.4)
(0.18706333107955828, 0.45)
(0.19473203410475032, 0.5)
(0.1936822688494351, 0.55)
(0.19885217930820537, 0.6000000000000001)
(0.20611941466468425, 0.6500000000000001)
(0.20233217774976364, 0.7000000000000001)
(0.19413919413919412, 0.7500000000000001)
(0.1978974400128401, 0.8)
(0.19786269430051814, 0.8500000000000001)
(0.20309303657638492, 0.9000000000000001)
(0.19704926231557887, 0.9500000000000001)


In [ ]:
model.save('mlcf-dp-model-2021-2021-images-latest.h5')

In [ ]:
# Lets load all the validation images 2021

val_images_path_ids = [] # This list will contain the absolute path of each image and their id
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

#Extract images path and images ids
for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  val_images_path_ids.append([path_to_image,image.split('.')[0]])

val_images_path_ids_df = pd.DataFrame(val_images_path_ids, columns=['image_path','image_id']) # Dataframe to use in the prediction process

100%|██████████| 500/500 [00:00<00:00, 180944.95it/s]


In [ ]:
# Load images using the same preprocessing method used in training
val_images_x = []
for idx, row in val_images_path_ids_df.iterrows():
  path_image = row['image_path']
  img = tf.keras.preprocessing.image.load_img(path = path_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img)/255 # Transform image to array of shape (input_shape), and normalize values by dividing them over 255
  val_images_x.append(img)

In [ ]:
# Prediction

val_images_preds = model.predict(np.array(val_images_x)) # Predict
val_images_preds[val_images_preds>=0.6500000000000001] = 1
val_images_preds[val_images_preds<0.6500000000000001] = 0

In [ ]:
# Transformation of transformed labels to actual concepts

val_labels_predicted = mlb.inverse_transform(val_images_preds) # Use the transformer that was used in the autoencoder model training

# Join predicted concepts and separate them by ;
val_labels_united = []
for prediction in val_labels_predicted:
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append(str_concepts[0:-1])

# The image id needs to be included in the submission
val_images_ids = []
for idx, row in val_images_path_ids_df.iterrows():
  val_images_ids.append(row['image_id'])

In [ ]:
# Create submission csv file that will contain the image_id \t concepts
final_predictions_val = pd.DataFrame({'image_ids': val_images_ids})
final_predictions_val['predictions'] = pd.Series(val_labels_united)
final_predictions_val.to_csv('/content/predictions-multilabel-classifier-using-all-images.csv', 
                             index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab

In [ ]:
with open("mlb_mlcf_dp_labels_2021_2020.pkl", 'wb') as f:
    pickle.dump(mlb, f)

## Using 2021 + 2020 data (without autoencoders) and only with bpo

In [ ]:
path_to_concepts = ['/content/images-combined-bpo-only.csv']

In [ ]:
def extract_concepts_all(root_paths, image_id_concepts_dict = dict()):
    for idx, name in enumerate(root_paths):
        with open(name, "r", encoding= 'utf-8-sig') as f:
          reader = csv.reader(f, delimiter = '\t')
          for i, line in enumerate(reader):
            if len(line[1])> 1:
              image_id_concepts_dict[line[0]] = list(line[1].split(";"))
            else:
              image_id_concepts_dict[line[0]] = []
    
    return image_id_concepts_dict

In [ ]:
#Extract concepts for the multiple concept files
image_id_concepts_dict = extract_concepts_all(path_to_concepts)

In [ ]:
len(image_id_concepts_dict)

83979

In [ ]:
# Since we are working with around 9K images. We will only load the images absolute path and the concepts to a dataframe and then use a generator to load them during training.
# Here, we will create a dataframe with the images path
all_images_path = []
# Training images
for image in tqdm(image_id_concepts_dict.keys(), position = 0):
  all_images_path.append([image])
df_all_images = pd.DataFrame(all_images_path, columns=['image_path'])

100%|██████████| 83979/83979 [00:00<00:00, 341298.26it/s]


In [ ]:
concepts = []
for image in tqdm(image_id_concepts_dict.keys(), position=0):
  concepts.append(image_id_concepts_dict[image])

100%|██████████| 83979/83979 [00:00<00:00, 1184447.80it/s]


In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
# Since we will use flow_from_dataframe in the training, we put both the images absolute path and the encoded labels
df_use_densenet = pd.concat([df_all_images, pd.DataFrame(np.array(mlb.fit_transform(concepts)))], axis=1)

In [ ]:
len(df_use_densenet)

83979

In [ ]:
len(mlb.classes_)

725

In [ ]:
concepts_binarized = np.array(mlb.transform(concepts))

In [ ]:
# Train split dataset, because a portion is needed to set the threshold (to see if to assign the concept or not) and another portion to see the overall f1-score
df_use_train, df_test = train_test_split(df_use_densenet, test_size = 0.05, shuffle = True, random_state = 14) # test will be used to get a final f1-score
df_train, df_val = train_test_split(df_use_train, test_size=0.05, shuffle=True, random_state=14)

In [ ]:
concepts_binarized_use_train, concepts_binarized_test = train_test_split(concepts_binarized, test_size = 0.05, shuffle = True, random_state = 14)
concepts_binarized_train, concepts_binarized_val = train_test_split(concepts_binarized_use_train, test_size = 0.05, shuffle = True, random_state = 14)

In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(75791, 726)
(3989, 726)
(4199, 726)


In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid', name = 'prediction_layer')(x)


model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x) # Final model to be trained

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 3, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Train generator
train_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=64, shuffle=True, seed=14, subset='training')

# Validation generator
val_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=64, shuffle=True, seed=14, subset='validation')

Found 60633 validated image filenames.
Found 15158 validated image filenames.


In [ ]:
layer_names = [layer.name for layer in default_densenet.layers]
layer_idx = layer_names.index('conv4_block7_0_bn')

In [ ]:
for layer in default_densenet.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Model training (of the entire model)
history_fined = model.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks)

In [ ]:
val_gen_pred = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Validation data generator for threshold tunning

val_generator_pred = val_gen_pred.flow_from_dataframe(df_val,x_col='image_path', y_col=df_val.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=64)

In [ ]:
# Since the predictions made by the model is a list of probabilities that a particular concept (from the one seen in training) is present, i.e [0.01, 0.5,...]
# It is needed to set a threshold for prob (0-1) that maximizes this f1 score.
probs = np.arange(0.05,1.0,0.05)
scores = []
for prob in probs:
  preds = model.predict(val_generator_pred) # Use the validation set
  preds[preds>=prob] = 1
  preds[preds<prob] = 0
  scores.append(tuple((f1_score(concepts_binarized_val, preds, average="micro"),prob)))
  print(tuple((f1_score(concepts_binarized_val, preds, average="micro"),prob)))

In [ ]:
model.save('mlcf-bpo-model-2021-2021-images-latest.h5')

In [ ]:
# Lets load all the validation images 2021

val_images_path_ids = [] # This list will contain the absolute path of each image and their id
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

#Extract images path and images ids
for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  val_images_path_ids.append([path_to_image,image.split('.')[0]])

val_images_path_ids_df = pd.DataFrame(val_images_path_ids, columns=['image_path','image_id']) # Dataframe to use in the prediction process

100%|██████████| 500/500 [00:00<00:00, 180944.95it/s]


In [ ]:
# Load images using the same preprocessing method used in training
val_images_x = []
for idx, row in val_images_path_ids_df.iterrows():
  path_image = row['image_path']
  img = tf.keras.preprocessing.image.load_img(path = path_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img)/255 # Transform image to array of shape (input_shape), and normalize values by dividing them over 255
  val_images_x.append(img)

In [ ]:
# Prediction

val_images_preds = model.predict(np.array(val_images_x)) # Predict
val_images_preds[val_images_preds>=0.6500000000000001] = 1
val_images_preds[val_images_preds<0.6500000000000001] = 0

In [ ]:
# Transformation of transformed labels to actual concepts

val_labels_predicted = mlb.inverse_transform(val_images_preds) # Use the transformer that was used in the autoencoder model training

# Join predicted concepts and separate them by ;
val_labels_united = []
for prediction in val_labels_predicted:
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append(str_concepts[0:-1])

# The image id needs to be included in the submission
val_images_ids = []
for idx, row in val_images_path_ids_df.iterrows():
  val_images_ids.append(row['image_id'])

In [ ]:
# Create submission csv file that will contain the image_id \t concepts
final_predictions_val = pd.DataFrame({'image_ids': val_images_ids})
final_predictions_val['predictions'] = pd.Series(val_labels_united)
final_predictions_val.to_csv('/content/predictions-multilabel-classifier-using-all-images-bpo.csv', 
                             index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab

In [ ]:
with open("mlb_mlcf_bpo_labels_2021_2020.pkl", 'wb') as f:
    pickle.dump(mlb, f)t

# Multi-label Classification with autoencoders

## Using 2021 + 2020 data (and using autoencoder trained using all concepts)

Right now, when we use MultilabelBinarizer to transform the labels, the final dimension of the vector is around 1585 length (1s and 0s). By using a autoencoder, we are trying to reduce the dimensionality of our labels (output), and what the autoencoder will do, is to learn how to take a set of 1s and 0s, and reproduce those 1s and 0s again. However, the layers within the autoencoder will have a lower dimension. From having a label of length 1585, the encoder will transform it into a vector of 100 (**ENCODED LABELS**), and the decoder will be in charge of transforming that reduce vector into the orriginal one.

### Preprocessing

In [ ]:
# Function to extract concepts from the training and validation images from 2021, and also from selected images from 2020 ImageCLEF dataset
# These selected images are images that strictly have the same concepts of this year dataset, therefore, 6,556 images from last year dataset are being used
def extract_concepts_all(root_paths, image_id_concepts_dict = dict()):
    """
      Function that extract concepts for a concept file (csv and json), and stores them in a dictionary, 
      where the key is the absolute path of the image and the values are the concepts.

      root_paths: a 1-d list that contains the absolute paths of the concept files
      image_id_concepts_dict: dictionary that will contain the data from the concepts file

      Returns: a dictionary with the absolute images paths as keys and their corresponding concepts as values.

    """
    for idx, name in enumerate(root_paths):
      if idx!=2:
        with open(name, "r", encoding= 'utf-8-sig') as f:
          reader = csv.reader(f, delimiter = '\t')
          if name =='/content/Training_Set_Concepts.csv':
            path_image = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'

          if name == '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation_Set_Concepts.csv':
            path_image = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'

          for i, line in enumerate(reader):
            if idx != 2:
              
              # It is recommended to check where the image has assigned concepts. This is relevant when separating concepts by sematic type
              # If the image does not have a concept, an empty list will be passed.
              if len(line[1]) < 1:
                image_id_concepts_dict[path_image+line[0]+'.jpg'] = []
              else:
                image_id_concepts_dict[path_image+line[0]+'.jpg'] = list(line[1].split(';'))
            else:
                if len(line[1]) < 1:
                  image_id_concepts_dict[line[0]] = []

                else:
                  image_id_concepts_dict[line[0]] = list(line[1].split(';'))
      else:
        # This section is strictly for the selected images from 2020 dataset
        images_2020 = json.load(open(name))
        for image in images_2020.keys():
          if len(images_2020[image]) > 8:
            image_id_concepts_dict[image] = images_2020[image].split(';')
          else:
            image_id_concepts_dict[image] = [images_2020[image]]


    return image_id_concepts_dict

In [ ]:
#Extract concepts for the multiple concept files
path_to_concepts = ['/content/Training_Set_Concepts.csv','/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation_Set_Concepts.csv',
                    '/content/images_2020_to_be_considered.json']
image_id_concepts_dict = extract_concepts_all(path_to_concepts)

In [ ]:
# Since we are working with around 9K images. We will only load the images absolute path and the concepts to a dataframe and then use a generator to load them during training.
# Here, we will create a dataframe with the images path
X = []
df_all_images_ids = pd.DataFrame(columns=['image_path'])
# Training images
for image in tqdm(image_id_concepts_dict.keys(), position = 0):
  X.append(image)
df_all_images_ids['image_path'] = X

100%|██████████| 9792/9792 [00:00<00:00, 1593737.86it/s]


In [ ]:
# Transforming and encoding process
# Since we need the encoded labels, we will use the transformer used in the autoencoder process

#Load transformer
with open("/content/mlb_autoencoder_all_labels.pkl", 'rb') as f:
    mlb = pickle.load(f)

# Put all concepts in a list of lists to be passed to the transformer
labels =[]
for image in tqdm(image_id_concepts_dict.keys(), position=0):
  labels.append(image_id_concepts_dict[image])

labels_transformed = mlb.transform(labels) # This will be used to get the encoded labels

# Load trained encoder
encoder = tf.keras.models.load_model('/content/encoder-all-combined-images.h5', compile=False)

# Encode transformed labels
Y = np.array(encoder.predict(labels_transformed))

In [ ]:
# Since we will use flow_from_dataframe in the training, we put both the images absolute path and the encoded labels
df_use_densenet = pd.concat([df_all_images_ids, pd.DataFrame(Y)], axis=1)

In [ ]:
# Train split dataset, because a portion is needed to set the threshold (to see if to assign the concept or not) and another portion to see the overall f1-score
df_train, df_test = train_test_split(df_use_densenet, test_size = 0.2, shuffle = True, random_state = 14) # test will be used to get a final f1-score
y_train, y_test = train_test_split(Y,test_size = 0.2, shuffle = True, random_state = 14)
labels_transformed_train, labels_transformed_test = train_test_split(labels_transformed,test_size = 0.2, shuffle = True, random_state = 14)

### Model training

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(100, activation='sigmoid', name = 'prediction_layer')(x)


model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x) # Final model to be trained

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 10, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model. Since the output is no longer an array of 1s and 0s, the loss function can change to a different one.
model.compile(loss = 'mean_squared_error', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Train generator
train_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=32, shuffle=True, seed=14, subset='training')

# Validation generator
val_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=32, shuffle=True, seed=14, subset='validation')


In [ ]:
# Model training (only the classification layers that have been added)
history = model.fit(train_generator, epochs = epochs, validation_data= val_generator, validation_steps = 20, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
196/196 [==============================] - 87s 265ms/step - loss: 0.3647 - acc: 0.0440 - val_loss: 0.2724 - val_acc: 0.1172
Epoch 2/100
196/196 [==============================] - 43s 221ms/step - loss: 0.2724 - acc: 0.1110 - val_loss: 0.2592 - val_acc: 0.1234
Epoch 3/100
196/196 [==============================] - 43s 221ms/step - loss: 0.2604 - acc: 0.1269 - val_loss: 0.2418 - val_acc: 0.1203
Epoch 4/100
196/196 [==============================] - 43s 220ms/step - loss: 0.2457 - acc: 0.1241 - val_loss: 0.2406 - val_acc: 0.1281
Epoch 5/100
196/196 [==============================] - 43s 222ms/step - loss: 0.2401 - acc: 0.1333 - val_loss: 0.2364 - val_acc: 0.1078
Epoch 6/100
196/196 [==============================] - 43s 221ms/step - loss: 0.2321 - acc: 0.1194 - val_loss: 0.2288 - val_acc: 0.1266
Epoch 7/100
196/196 [==============================] - 44s 222ms/step - loss: 0.2321 - acc: 0.1219 - val_loss: 0.2315 - val_acc: 0.1016
Epoch 8/100
196/196 [===========================

In [ ]:
# Now that our classification layer has been trained, we can unfreeze the rest of the model, which are the convolutional blocks
default_densenet.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=new_lr, decay=new_lr / epochs)

In [ ]:
# Compile model
model.compile(loss = 'mean_squared_error', optimizer=opt, metrics=['acc'])

In [ ]:
# Model training (of the entire model)
history_fined = model.fit(train_generator, epochs = epochs, validation_data= val_generator, validation_steps = 20, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
196/196 [==============================] - 86s 380ms/step - loss: 0.2214 - acc: 0.2139 - val_loss: 0.1949 - val_acc: 0.3094
Epoch 2/100
196/196 [==============================] - 72s 366ms/step - loss: 0.1939 - acc: 0.3054 - val_loss: 0.1893 - val_acc: 0.3547
Epoch 3/100
196/196 [==============================] - 73s 371ms/step - loss: 0.1776 - acc: 0.3165 - val_loss: 0.1859 - val_acc: 0.3125
Epoch 4/100
196/196 [==============================] - 73s 371ms/step - loss: 0.1721 - acc: 0.3485 - val_loss: 0.1932 - val_acc: 0.3266
Epoch 5/100
196/196 [==============================] - 73s 371ms/step - loss: 0.1715 - acc: 0.3302 - val_loss: 0.1822 - val_acc: 0.3266
Epoch 6/100
196/196 [==============================] - 73s 371ms/step - loss: 0.1632 - acc: 0.3607 - val_loss: 0.1775 - val_acc: 0.3141
Epoch 7/100
196/196 [==============================] - 73s 370ms/step - loss: 0.1612 - acc: 0.3637 - val_loss: 0.1824 - val_acc: 0.3141
Epoch 8/100
196/196 [===========================

### Evaluate the model on unseen data

In [ ]:
# Test data generator (same process that it was used in the training generators)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Here, the test split is being used, even though a y_col is being defined, it wont be used when predicting
test_generator = test_gen.flow_from_dataframe(df_test,x_col='image_path', y_col=df_test.columns[1:], target_size=(224,224),
                                                     class_mode ='raw', shuffle = False)

In [ ]:
# Predictions
predictions = model.predict(test_generator)

In [ ]:
# Decoding predictions
# In the preprocessing part, the encoder was used. Now, those encoded predictions need to be decoded into 1s and 0s learned by the autoencoder

decoder = tf.keras.models.load_model('/content/decoder-all-combined-images.h5', compile=False) # Load decoder
decoded_predictions = decoder.predict(predictions) # Decode predictions

# In the process of training the autoencoder, a threshold was tuned to decide what is the value to consider when setting the predictions of the decoder to 1s and 0s
# When tunning this value, 0.35... was the one with highest f1 score

decoded_predictions[decoded_predictions>=0.35000000000000003] = 1
decoded_predictions[decoded_predictions<0.35000000000000003] = 0

In [ ]:
# Compute f1-score
# A higher f1-score is expected for this, because of combining all the images (train and val 2021 images). However, this score is using unseen data
test_f1_score = f1_score(labels_transformed_test, decoded_predictions, average="micro")
print('F1-score (on test set): ' + str(test_f1_score))

F1-score (on test set): 0.6639718346590169


In [ ]:
# Save model
model.save('/content/multilabel-classifier-using-autoencoder-all-smt.h5')

### Create a submission file for evaluation (using evaluate-f1.py script)

In [ ]:
# Lets load all the validation images 2021

val_images_path_ids = [] # This list will contain the absolute path of each image and their id
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

#Extract images path and images ids
for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  val_images_path_ids.append([path_to_image,image.split('.')[0]])

val_images_path_ids_df = pd.DataFrame(val_images_path_ids, columns=['image_path','image_id']) # Dataframe to use in the prediction process

100%|██████████| 500/500 [00:00<00:00, 256312.88it/s]


In [ ]:
# Load images using the same preprocessing method used in training
val_images_x = []
for idx, row in val_images_path_ids_df.iterrows():
  path_image = row['image_path']
  img = tf.keras.preprocessing.image.load_img(path = path_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img)/255 # Transform image to array of shape (input_shape), and normalize values by dividing them over 255
  val_images_x.append(img)

In [ ]:
# Prediction

val_images_preds = model.predict(np.array(val_images_x)) # Predict
decoded_val_predictions = decoder.predict(val_images_preds) # Decode
decoded_val_predictions[decoded_val_predictions>=0.35000000000000003] = 1
decoded_val_predictions[decoded_val_predictions<0.35000000000000003] = 0

In [ ]:
# Transformation of transformed labels to actual concepts

val_labels_predicted = mlb.inverse_transform(decoded_val_predictions) # Use the transformer that was used in the autoencoder model training

# Join predicted concepts and separate them by ;
val_labels_united = []
for prediction in val_labels_predicted:
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append(str_concepts[0:-1])

# The image id needs to be included in the submission
val_images_ids = []
for idx, row in val_images_path_ids_df.iterrows():
  val_images_ids.append(row['image_id'])

In [ ]:
# Create submission csv file that will contain the image_id \t concepts
final_predictions_val = pd.DataFrame({'image_ids': val_images_ids})
final_predictions_val['predictions'] = pd.Series(val_labels_united)
final_predictions_val.to_csv('/content/predictions-multilabel-classifier-using-autoencoder-all-labels-v1.csv', 
                             index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab

### Using trained model to generate features for images

In [ ]:
auto_encoded_mlcf = tf.keras.models.load_model('/content/multilabel-classifier-using-autoencoder-all-smt.h5', compile=False)

In [ ]:
get_layer_output = tf.keras.backend.function([auto_encoded_mlcf.layers[0].input],[auto_encoded_mlcf.layers[-2].output])

In [ ]:
def feedForward_finetuned(fname,get_layer_output):

    img = tf.keras.preprocessing.image.load_img(fname, target_size=(224,224))
    x = tf.keras.preprocessing.image.img_to_array(img) / 255
    x = np.expand_dims(x, axis = 0)
    features = get_layer_output([x])[0]
    features = features.flatten()

    return features

def extract_store_finetuned(path_dir, model, additional_images = None):
  """
    Function that takes a path and passes all the images (inside that directory) to the feedForward function for feature extraction, and then creates
    a dataframe containing all features and image ids of the images.
    path_dir: full path of the directory that will be searched
    model: model to be used for feature extraction

    Returns: A pandas DataFrame containing all images and their corresponding features
  """
  id_features_vector = []
  print('Extracting features...')

  for image in tqdm(os.listdir(path_dir), position= 0, leave = False):
    path = os.path.join(path_dir, image)
    name_image = os.path.splitext(image)[0]
    image_id = int(name_image.replace('synpic',''))

    if additional_images == None:
      vector = feedForward_finetuned(path, model)
      vector = np.insert(vector,0,image_id)
      id_features_vector.append(vector)
    else:
      check_year = [int(2021)]
      year_id_vector = np.insert(check_year,0, image_id)
      vector = feedForward_finetuned(path, model)
      vector = np.insert(vector,0, year_id_vector)
      id_features_vector.append(vector)
  
  if additional_images != None:
    images_2020 = json.load(open(additional_images))
    check_year = [int(2020)]
    for image in images_2020.keys():
      path = image
      image_id = int(image.split("/")[-1].split(".")[0].split("_")[-1])
      year_id_vector = np.insert(check_year,0,image_id)
      print(year_id_vector)
      vector = feedForward_finetuned(path, model)
      vector = np.insert(vector, 0, year_id_vector)
      id_features_vector.append(vector)
  else:
    pass
      
  return id_features_vector

In [ ]:
# Training set (2021)
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images', 
                                             get_layer_output, additional_images = '/content/images_2020_to_be_considered.json')


#np.save('/content/features-224-densenet-fined-autoencoded-training-2021.npy', id_features_vector)


  0%|          | 1/2756 [00:00<07:14,  6.34it/s]

Extracting features...


Se truncaron las últimas líneas 5000 del resultado de transmisión.
[1232 2020]
[1234 2020]
[1236 2020]
[1258 2020]
[1278 2020]
[1281 2020]
[1287 2020]
[1288 2020]
[1294 2020]
[1299 2020]
[1313 2020]
[1319 2020]
[1324 2020]
[1332 2020]
[1334 2020]
[1355 2020]
[1365 2020]
[1373 2020]
[1375 2020]
[1377 2020]
[1399 2020]
[1402 2020]
[1449 2020]
[1453 2020]
[1458 2020]
[1474 2020]
[1477 2020]
[1487 2020]
[1488 2020]
[1495 2020]
[1507 2020]
[1518 2020]
[1519 2020]
[1545 2020]
[1593 2020]
[1595 2020]
[1619 2020]
[1622 2020]
[1639 2020]
[1643 2020]
[1659 2020]
[1680 2020]
[1694 2020]
[1705 2020]
[1710 2020]
[1712 2020]
[1721 2020]
[1745 2020]
[1747 2020]
[1756 2020]
[1774 2020]
[1780 2020]
[1783 2020]
[1787 2020]
[1794 2020]
[1808 2020]
[1821 2020]
[1822 2020]
[1829 2020]
[1842 2020]
[1848 2020]
[1855 2020]
[1860 2020]
[1870 2020]
[1919 2020]
[1920 2020]
[1981 2020]
[1983 2020]
[1985 2020]
[1986 2020]
[1995 2020]
[1996 2020]
[2001 2020]
[2010 2020]
[2013 2020]
[2021 2020]
[2034 2020]
[2037 202

In [ ]:
np.save('/content/features-224-densenet-fined-autoencoded-training-2021-2020.npy', id_features_vector)

In [ ]:
# Validation set (2021)
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images', get_layer_output)
np.save('/content/features-224-densenet-fined-validation-2021.npy', id_features_vector)

  0%|          | 1/500 [00:00<01:19,  6.26it/s]

Extracting features...


## Using 2021 + 2020 data (and using autoencoder trained using only diagnostic procedure concepts)

### Preprocessing

In [ ]:
path_to_concepts = ['/content/training-images-concepts-by-semantic/concepts-file/training-concepts-dp-only.csv',
                    '/content/val-concepts-dp-only.csv',
                    '/content/images-2020-dp-only.csv']

In [ ]:
# Function to extract concepts from the training and validation images from 2021, and also from selected images from 2020 ImageCLEF dataset
# These selected images are images that strictly have the same concepts of this year dataset, therefore, 6,556 images from last year dataset are being used
def extract_concepts_all(root_paths, image_id_concepts_dict = dict()):
    """
      Function that extract concepts for a concept file (csv and json), and stores them in a dictionary, 
      where the key is the absolute path of the image and the values are the concepts.

      root_paths: a 1-d list that contains the absolute paths of the concept files
      image_id_concepts_dict: dictionary that will contain the data from the concepts file

      Returns: a dictionary with the absolute images paths as keys and their corresponding concepts as values.

    """
    for idx, name in enumerate(root_paths):
        with open(name, "r", encoding= 'utf-8-sig') as f:
          reader = csv.reader(f, delimiter = '\t')
          if name =='/content/training-images-concepts-by-semantic/concepts-file/training-concepts-dp-only.csv':
            path_image = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'

          if name == '/content/val-concepts-dp-only.csv':
            path_image = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'

          for i, line in enumerate(reader):
            if idx != 2:
              # It is recommended to check where the image has assigned concepts. This is relevant when separating concepts by sematic type
              # If the image does not have a concept, an empty list will be passed.
              if len(line[1]) < 1:
                image_id_concepts_dict[path_image+line[0]+'.jpg'] = []
              else:
                image_id_concepts_dict[path_image+line[0]+'.jpg'] = list(line[1].split(';'))
            else:
                if len(line[1]) < 1:
                  image_id_concepts_dict[line[0]] = []

                else:
                  image_id_concepts_dict[line[0]] = list(line[1].split(';'))

    return image_id_concepts_dict

In [ ]:
#Extract concepts for the multiple concept files
image_id_concepts_dict = extract_concepts_all(path_to_concepts)

In [ ]:
# Since we are working with around 9K images. We will only load the images absolute path and the concepts to a dataframe and then use a generator to load them during training.
# Here, we will create a dataframe with the images path
X = []
df_all_images_ids = pd.DataFrame(columns=['image_path'])
# Training images
for image in tqdm(image_id_concepts_dict.keys(), position = 0):
  X.append(image)
df_all_images_ids['image_path'] = X

100%|██████████| 9792/9792 [00:00<00:00, 1412964.01it/s]


In [ ]:
# Transforming and encoding process
# Since we need the encoded labels, we will use the transformer used in the autoencoder process

#Load transformer
with open("/content/mlb_autoencoder_dp_labels.pkl", 'rb') as f:
    mlb = pickle.load(f)

# Put all concepts in a list of lists to be passed to the transformer
labels =[]
for image in tqdm(image_id_concepts_dict.keys(), position=0):
  labels.append(image_id_concepts_dict[image])

labels_transformed = mlb.transform(labels) # This will be used to get the encoded labels

# Load trained encoder
encoder = tf.keras.models.load_model('/content/encoder-dp-combined-images.h5', compile=False)

# Encode transformed labels
Y = np.array(encoder.predict(labels_transformed))

100%|██████████| 9792/9792 [00:00<00:00, 1215071.29it/s]


In [ ]:
labels[0:10]

[['C0024485'],
 ['C0032743'],
 ['C0040398'],
 ['C0024485'],
 ['C2456881', 'C0041618'],
 ['C0040398'],
 ['C0040398'],
 [],
 ['C0412611', 'C0040398'],
 ['C0040398']]

In [ ]:
labels_transformed[0:2]

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
Y[0:2]

array([[0.        , 2.2402322 , 0.6117097 , 0.        , 3.6497285 ,
        0.63613445, 3.4092917 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 2.4441638 , 3.8091717 ,
        5.4655886 , 0.        , 2.3914764 , 2.8667312 , 1.9693022 ],
       [0.        , 2.4963117 , 3.5909462 , 0.        , 2.0191474 ,
        1.0522851 , 3.1866229 , 0.20849384, 0.40269482, 2.0956373 ,
        0.        , 0.        , 0.        , 1.9132146 , 0.96986383,
        2.2350454 , 0.        , 0.10135684, 1.4293385 , 2.3868608 ]],
      dtype=float32)

In [ ]:
# Since we will use flow_from_dataframe in the training, we put both the images absolute path and the encoded labels
df_use_densenet = pd.concat([df_all_images_ids, pd.DataFrame(Y)], axis=1)

In [ ]:
# Train split dataset, because a portion is needed to set the threshold (to see if to assign the concept or not) and another portion to see the overall f1-score
df_train, df_test = train_test_split(df_use_densenet, test_size = 0.2, shuffle = True, random_state = 14) # test will be used to get a final f1-score
y_train, y_test = train_test_split(Y,test_size = 0.2, shuffle = True, random_state = 14)
labels_transformed_train, labels_transformed_test = train_test_split(labels_transformed,test_size = 0.2, shuffle = True, random_state = 14)

### Model training

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet') # Load model (only feature extraction part) with imagenet weights
default_densenet.trainable = False # Freeze all layers of the model, so weights remain the same when training, and only weights from added layers update

In [ ]:
# Adding the classification part to the existing model
x = tf.keras.layers.GlobalAveragePooling2D()(default_densenet.output)
x = tf.keras.layers.Dense(20, activation='sigmoid', name = 'prediction_layer')(x)


model = tf.keras.models.Model(inputs = default_densenet.input, outputs= x) # Final model to be trained

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 10, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model. Since the output is no longer an array of 1s and 0s, the loss function can change to a different one.
model.compile(loss = 'mean_squared_error', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Train generator
train_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=32, shuffle=True, seed=14, subset='training')

# Validation generator
val_generator = data_generator.flow_from_dataframe(df_train,x_col='image_path', y_col=df_train.columns[1:], target_size=(224,224),
                                                     class_mode ='raw',batch_size=32, shuffle=True, seed=14, subset='validation')


Found 6267 validated image filenames.
Found 1566 validated image filenames.


In [ ]:
# Model training (only the classification layers that have been added)
history = model.fit(train_generator, epochs = epochs, validation_data= val_generator, validation_steps = 20, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
196/196 [==============================] - 57s 265ms/step - loss: 2.3288 - acc: 0.1240 - val_loss: 1.8826 - val_acc: 0.1656
Epoch 2/100
196/196 [==============================] - 44s 222ms/step - loss: 1.9117 - acc: 0.1789 - val_loss: 1.8798 - val_acc: 0.1922
Epoch 3/100
196/196 [==============================] - 43s 220ms/step - loss: 1.8849 - acc: 0.1908 - val_loss: 1.8420 - val_acc: 0.1922
Epoch 4/100
196/196 [==============================] - 43s 220ms/step - loss: 1.8676 - acc: 0.1895 - val_loss: 1.8477 - val_acc: 0.1781
Epoch 5/100
196/196 [==============================] - 43s 219ms/step - loss: 1.8729 - acc: 0.1953 - val_loss: 1.8875 - val_acc: 0.1703
Epoch 6/100
196/196 [==============================] - 43s 220ms/step - loss: 1.8557 - acc: 0.1914 - val_loss: 1.8562 - val_acc: 0.1844
Epoch 7/100
196/196 [==============================] - 43s 219ms/step - loss: 1.8727 - acc: 0.1988 - val_loss: 1.8424 - val_acc: 0.1984
Epoch 8/100
196/196 [===========================

In [ ]:
# Now that our classification layer has been trained, we can unfreeze the rest of the model, which are the convolutional blocks
default_densenet.trainable = True

In [ ]:
# A new learning rate is defined, since a keras guide (https://keras.io/guides/transfer_learning/) suggests to lower it. Search for "It's also critical to use a very low learning"
new_lr = 1e-5

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=new_lr)

In [ ]:
# Compile model
model.compile(loss = 'mean_squared_error', optimizer=opt, metrics=['acc'])

In [ ]:
# Model training (of the entire model)
history_fined = model.fit(train_generator, epochs = epochs, validation_data= val_generator, validation_steps = 20, verbose= 1,
                               callbacks = callbacks)

Epoch 1/100
196/196 [==============================] - 86s 382ms/step - loss: 1.8634 - acc: 0.2161 - val_loss: 1.8312 - val_acc: 0.1688
Epoch 2/100
196/196 [==============================] - 72s 368ms/step - loss: 1.8368 - acc: 0.2057 - val_loss: 1.8450 - val_acc: 0.1641
Epoch 3/100
196/196 [==============================] - 73s 371ms/step - loss: 1.8416 - acc: 0.2112 - val_loss: 1.8198 - val_acc: 0.2016
Epoch 4/100
196/196 [==============================] - 73s 374ms/step - loss: 1.8206 - acc: 0.2089 - val_loss: 1.8286 - val_acc: 0.1984
Epoch 5/100
196/196 [==============================] - 74s 375ms/step - loss: 1.8256 - acc: 0.2020 - val_loss: 1.8240 - val_acc: 0.1922
Epoch 6/100
196/196 [==============================] - 74s 374ms/step - loss: 1.8170 - acc: 0.2111 - val_loss: 1.8052 - val_acc: 0.2047
Epoch 7/100
196/196 [==============================] - 73s 374ms/step - loss: 1.8220 - acc: 0.2213 - val_loss: 1.7879 - val_acc: 0.2484
Epoch 8/100
196/196 [===========================

### Evaluate the model on unseen data

In [ ]:
# Test data generator (same process that it was used in the training generators)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Here, the test split is being used, even though a y_col is being defined, it wont be used when predicting
test_generator = test_gen.flow_from_dataframe(df_test,x_col='image_path', y_col=df_test.columns[1:], target_size=(224,224),
                                                     class_mode ='raw', shuffle = False)

Found 1959 validated image filenames.


In [ ]:
# Predictions
predictions = model.predict(test_generator)

In [ ]:
predictions[0]

array([0.9994357 , 0.01967256, 0.754381  , 0.0019035 , 0.9999232 ,
       0.6805687 , 0.02403776, 0.0148264 , 0.37443423, 0.9999558 ,
       0.9128858 , 0.00334915, 0.0033653 , 0.99997044, 0.99985945,
       0.99984026, 0.00129978, 0.9999738 , 0.14336427, 0.9998945 ],
      dtype=float32)

In [ ]:
# Decoding predictions
# In the preprocessing part, the encoder was used. Now, those encoded predictions need to be decoded into 1s and 0s learned by the autoencoder

decoder = tf.keras.models.load_model('/content/decoder-dp-combined-images.h5', compile=False) # Load decoder
decoded_predictions = decoder.predict(predictions) # Decode predictions

In [ ]:
# In the process of training the autoencoder, a threshold was tuned to decide what is the value to consider when setting the predictions of the decoder to 1s and 0s
# When tunning this value, 0.35... was the one with highest f1 score

decoded_predictions[decoded_predictions>=0.4] = 1
decoded_predictions[decoded_predictions<0.4] = 0

In [ ]:
# Compute f1-score
# A higher f1-score is expected for this, because of combining all the images (train and val 2021 images). However, this score is using unseen data
test_f1_score = f1_score(labels_transformed_test, decoded_predictions, average="micro")
print('F1-score (on test set): ' + str(test_f1_score))

F1-score (on test set): 0.6674692993017097


In [ ]:
# Save model
model.save('/content/multilabel-classifier-using-autoencoder-all-smt.h5')

### Create a submission file for evaluation (using evaluate-f1.py script)

In [ ]:
# Lets load all the validation images 2021

val_images_path_ids = [] # This list will contain the absolute path of each image and their id
validation_images_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'

#Extract images path and images ids
for image in tqdm(os.listdir(validation_images_path), position= 0):
  path_to_image = os.path.join(validation_images_path, image)
  val_images_path_ids.append([path_to_image,image.split('.')[0]])

val_images_path_ids_df = pd.DataFrame(val_images_path_ids, columns=['image_path','image_id']) # Dataframe to use in the prediction process

In [ ]:
# Load images using the same preprocessing method used in training
val_images_x = []
for idx, row in val_images_path_ids_df.iterrows():
  path_image = row['image_path']
  img = tf.keras.preprocessing.image.load_img(path = path_image, target_size = (224,224)) # Load actual image
  img = tf.keras.preprocessing.image.img_to_array(img)/255 # Transform image to array of shape (input_shape), and normalize values by dividing them over 255
  val_images_x.append(img)

In [ ]:
# Prediction

val_images_preds = model.predict(np.array(val_images_x)) # Predict
decoded_val_predictions = decoder.predict(val_images_preds) # Decode
decoded_val_predictions[decoded_val_predictions>=0.4] = 1
decoded_val_predictions[decoded_val_predictions<0.4] = 0

In [ ]:
# Transformation of transformed labels to actual concepts

val_labels_predicted = mlb.inverse_transform(decoded_val_predictions) # Use the transformer that was used in the autoencoder model training

# Join predicted concepts and separate them by ;
val_labels_united = []
for prediction in val_labels_predicted:
  str_concepts = ''
  for concept in prediction:
    str_concepts += concept+';'
  val_labels_united.append(str_concepts[0:-1])

# The image id needs to be included in the submission
val_images_ids = []
for idx, row in val_images_path_ids_df.iterrows():
  val_images_ids.append(row['image_id'])

In [ ]:
# Create submission csv file that will contain the image_id \t concepts
final_predictions_val = pd.DataFrame({'image_ids': val_images_ids})
final_predictions_val['predictions'] = pd.Series(val_labels_united)
final_predictions_val.to_csv('/content/predictions-multilabel-classifier-using-autoencoder-dp-only.csv', 
                             index= False, sep ='\t', header= False) # Dont include headers, and image_id and concepts need to be separated by tab